In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
# random fake dataset
da = xr.DataArray(np.random.randn(2, 3, 2), dims=("x", "y",'t'), coords={"x": [10, 20], 'y':[33,44,55], 't':[7,8]})
da=da.rename('elev')
ds=da.to_dataset()

In [3]:
ds

<xarray.Dataset>
Dimensions:  (t: 2, x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 33 44 55
  * t        (t) int64 7 8
Data variables:
    elev     (x, y, t) float64 -0.09118 -0.528 0.03495 ... -0.449 1.014 -0.1689

In [4]:
# function to apply
def fn(x,y,elev,z):
    misc_arr = [np.array([[1,2],[3,4],[5,6],[1,2]]), np.array([[9,8],[7,6],[5,4],[3,2],[1,0],[9,8]])]
    return misc_arr

In [5]:
# using groupby and apply --> this is likely a bug that was fixed in the most recent version
ds.groupby('t', squeeze=False).apply(fn, args=(ds.x, ds.y, ds.elev, 10))

TypeError: fn() takes 4 positional arguments but 5 were given

In [6]:
# create a groupby wrapper to deal with the args issues
def fn_wrapper(gb):
    x=gb.x
    y=gb.y
    elev=gb.elev
    z = 12

#     print(gb)
    array = fn(x,y,elev,z)
    bergs=pd.Series({'bergs':[array]}, dtype='object')
    return gb.assign(new_var=('t',bergs))

In [7]:
ds.groupby('t', squeeze=False).apply(fn_wrapper)

<xarray.Dataset>
Dimensions:  (t: 2, x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 33 44 55
  * t        (t) int64 7 8
Data variables:
    elev     (x, y, t) float64 -0.09118 -0.528 0.03495 ... -0.449 1.014 -0.1689
    new_var  (t) object [[array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]])]] [[array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]])]]

In [8]:
# this doesn't work because the function itself still returns the array. It's the wrapper that turns the array into a series with dtype object to add back in to the DataSet
# could add an xarray boolean flag to the fn that defaults to false but returns the series instead if run for an xarray object
# xr.apply_ufunc(fn, ds.x, ds.y, ds.elev.groupby('t'),12, input_core_dims=[['x'],['y'],['x','y'],[]],output_core_dims=[['t']], exclude_dims=set(['x','y']))
new_vals=xr.apply_ufunc(fn, ds.x, ds.y, ds.elev.groupby('t',squeeze=False), 12, input_core_dims=[['x'],['y'],['x','y','t'],[]],output_core_dims=[['t']], exclude_dims=set(['x','y']))

ValueError: size of dimension 't' on inputs was unexpectedly changed by applied function from 1 to 2. Only dimensions specified in ``exclude_dims`` with xarray.apply_ufunc are allowed to change size.

In [144]:
new_vals

<xarray.DataArray (t: 2)>
array([array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]]),
       array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]])], dtype=object)
Coordinates:
  * t        (t) int64 7 8

In [147]:
xr.merge([ds,new_vals.rename('new_val')])

<xarray.Dataset>
Dimensions:  (t: 2, x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 33 44 55
  * t        (t) int64 7 8
Data variables:
    elev     (x, y, t) float64 1.725 0.4116 -1.056 ... 0.6464 1.252 0.5269
    new_val  (t) object [[1 2]
 [3 4]
 [5 6]
 [1 2]] [[9 8]
 [7 6]
 [5 4]
 [3 2]
 [1 0]
 [9 8]]

In [204]:
# prepending the variable doesn't work because the dataset is rewritten during the groupby().apply() operation.
# first add the new variable to the dataset before running the function
ds=ds.assign(new_var=('t',[misc_arr, misc_arr2]))
ds

<xarray.Dataset>
Dimensions:  (t: 2, x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 33 44 55
  * t        (t) int64 7 8
Data variables:
    elev     (x, y, t) float64 1.725 0.4116 -1.056 ... 0.6464 1.252 0.5269
    new_var  (t) object [array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]])] [array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]]), array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]])]

## Is what I want to do possible? YES

***DO NOTE DELETE THESE BLOCKS OF CODE - USE FOR EXAMPLE!***

In [203]:
misc_arr = [np.array([[1,2],[3,4],[5,6],[1,2]]), np.array([[9,8],[7,6],[5,4],[3,2],[1,0],[9,8]])]
misc_arr2 = [np.array([[9,8],[7,6],[5,4],[3,2],[1,0],[9,8]]), np.array([[9,8],[7,6],[5,4],[3,2],[1,0],[9,8]]), 
             np.array([[1,2],[3,4],[5,6],[1,2]])]
misc_arr3 = [np.array([[1,2],[3,4],[5,6],[7,8],[1,2]])]
misc_arr4 = [np.array([[1,2],[3,4],[5,6],[1,2]]), np.array([[9,8],[7,6],[5,4],[9,8]]), 
             np.array([[1,2],[3,4],[5,6],[7,6],[5,4],[1,2]]), np.array([[3,4],[5,6],[10,12],[9,8],[1,2]]),
             np.array([[9,8],[7,6],[5,4],[1,2],[3,4],[9,8]])]

print(len(misc_arr))
print(len(misc_arr2))
print(len(misc_arr3))
print(len(misc_arr4))

In [76]:
da = xr.DataArray([misc_arr, misc_arr2, misc_arr3, misc_arr4], dims='time', name='tests')
da

<xarray.DataArray 'tests' (time: 4)>
array([list([array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]])]),
       list([array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]]), array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]])]),
       list([array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 8],
       [1, 2]])]),
       list([array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [9, 8]]), array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 6],
       [5, 4],
       [1, 2]]), array([[ 3,  4],
       [ 5,  6],
       [10, 12],
       [ 9,  8],
       [ 1,  2]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [1, 2],
       [3, 4],
       [9, 8]])])], dtype=object)
Dimensions without coordinates: time

In [44]:
ds = da.to_dataset()
ds

<xarray.Dataset>
Dimensions:  (time: 4)
Dimensions without coordinates: time
Data variables:
    tests    (time) object [array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [3, 2],
       [1, 0],
       [9, 8]])] ... [array([[1, 2],
       [3, 4],
       [5, 6],
       [1, 2]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [9, 8]]), array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 6],
       [5, 4],
       [1, 2]]), array([[ 3,  4],
       [ 5,  6],
       [10, 12],
       [ 9,  8],
       [ 1,  2]]), array([[9, 8],
       [7, 6],
       [5, 4],
       [1, 2],
       [3, 4],
       [9, 8]])]

Helpful XArray resources from Ryan and others

http://xarray.pydata.org/en/stable/indexing.html#align-and-reindex

http://xarray.pydata.org/en/stable/examples/multidimensional-coords.html#Plotting

https://earth-env-data-science.github.io/lectures/xarray/xarray-part2.html

https://earth-env-data-science.github.io/lectures/xarray/xarray-part2.html#transformations
https://earth-env-data-science.github.io/lectures/pandas/pandas_groupby.html


https://rabernat.github.io/research_computing_2018/xarray.html

https://corteva.github.io/rioxarray/stable/examples/reproject_match.html